In [319]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [320]:
# Lista z plikami csv
csv_file = ['EPL_2021.csv', 'EPL_2020.csv', 'EPL_2019.csv', 'EPL_2018.csv', 'EPL_2017.csv']

In [321]:
df_concat = pd.concat([pd.read_csv(f) for f in csv_file], ignore_index=True)

In [322]:
df_concat.shape

(3800, 59)

In [323]:
#Czyszczenie zbioru
df_concat.drop(columns=['Comp', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes', 'xG', 'xGA'], inplace = True)


In [324]:
#df_concat.head(5)
df_concat.shape

(3800, 51)

In [325]:
#Wypełnianie brakujących przestrzeni
df_concat['Dist'].fillna(value=df_concat['Dist'].mean(), inplace=True)
df_concat['Attendance'].fillna(0 , inplace=True)

In [326]:
#Zamiana typu danych

#Drużyny LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_concat[['Opponent', 'Team']] = df_concat[['Opponent', 'Team']].apply(le.fit_transform)
teams_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(teams_mapping)

#Zamiana dni na int
df_concat['Day'] = df_concat['Day'].replace(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],[1, 2, 3, 4, 5, 6, 7])

#Zamiana rezultatu na int
df_concat['Result'] = df_concat['Result'].replace(['L','D','W'],[0 , 1, 2])

#Zamiana Dom/Wyjazd na int
df_concat['Venue'] = df_concat['Venue'].replace(['Away', 'Home'],[1 , 0])

#Zamiana daty Object na datetime
df_concat['Date'] = pd.to_datetime(df_concat['Date'])

#Zamiana czasu na godzinę 
df_concat['Time'] = pd.to_datetime(df_concat['Time'], format='%H:%M').dt.time
df_concat['Time'] = pd.to_datetime(df_concat['Time'], format='%H:%M:00').dt.hour

#Zamina czasu na kolejkę
df_concat['Round'] = df_concat['Round'].replace(['Matchweek 1', 'Matchweek 2', 'Matchweek 3', 'Matchweek 4', 'Matchweek 5',
 'Matchweek 6', 'Matchweek 7', 'Matchweek 8', 'Matchweek 9', 'Matchweek 10',
 'Matchweek 11', 'Matchweek 12', 'Matchweek 13', 'Matchweek 14',
 'Matchweek 15', 'Matchweek 16', 'Matchweek 17', 'Matchweek 18',
 'Matchweek 19', 'Matchweek 20', 'Matchweek 21', 'Matchweek 22',
 'Matchweek 23', 'Matchweek 24', 'Matchweek 25', 'Matchweek 26',
 'Matchweek 27', 'Matchweek 28', 'Matchweek 29', 'Matchweek 31',
 'Matchweek 32', 'Matchweek 30', 'Matchweek 34', 'Matchweek 35',
 'Matchweek 36', 'Matchweek 33', 'Matchweek 37', 'Matchweek 38'],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37 ,38 ])

 #df_concat.dtypes OK




{'Arsenal': 0, 'Aston Villa': 1, 'Bournemouth': 2, 'Brentford': 3, 'Brighton and Hove Albion': 4, 'Burnley': 5, 'Cardiff City': 6, 'Chelsea': 7, 'Crystal Palace': 8, 'Everton': 9, 'Fulham': 10, 'Huddersfield Town': 11, 'Leeds United': 12, 'Leicester City': 13, 'Liverpool': 14, 'Manchester City': 15, 'Manchester United': 16, 'Newcastle United': 17, 'Norwich City': 18, 'Sheffield United': 19, 'Southampton': 20, 'Stoke City': 21, 'Swansea City': 22, 'Tottenham Hotspur': 23, 'Watford': 24, 'West Bromwich Albion': 25, 'West Ham United': 26, 'Wolverhampton Wanderers': 27}


In [327]:
#Dodanie zmiennych średnich w przeciągu ostatnich pięciu meczy

def rolling_averages(matches_by_team, columns, new_columns):
    matches_by_team = matches_by_team.sort_values("Date")
    rolling_stats = matches_by_team[cols].rolling(5, closed='left').mean()
    matches_by_team[new_columns] = rolling_stats
    matches_by_team = matches_by_team.dropna(subset=new_columns)
    return matches_by_team

In [328]:
columns = ['Result', 'GF', 'GA', 'Poss', 'Attendance', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt', 'SoTA', 'PKA', 'PKsv', 'PKm' , 'Cmp_Total', 'Att_Total', 'TotDist', 'PrgDist', 'TI', 'CK', 'SCA', 'GCA', 'Tkl_Defensive',	'TklW_Defensive', 'Blocks_Defensive', 'Touches', 'Def Pen',	'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Dribbles_Succ', 'Dribles_Att', 'Rec',	'CrdY',	'CrdR',	'2CrdY'	, 'Fls', 'Fld',	'Off', 'PKwon',	'PKcon', 'OG']
new_columns = [f"{c}_rolling" for c in columns]

In [329]:
matches_rolling = df_concat.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling = matches_rolling.droplevel("Team")
matches_rolling.drop(columns=['Result', 'GF', 'GA', 'Poss', 'Attendance', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt', 'SoTA', 'PKA', 'PKsv', 'PKm' , 'Cmp_Total', 'Att_Total', 'TotDist', 'PrgDist', 'TI', 'CK', 'SCA', 'GCA', 'Tkl_Defensive',	'TklW_Defensive', 'Blocks_Defensive', 'Touches', 'Def Pen',	'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Dribbles_Succ', 'Dribles_Att', 'Rec',	'CrdY',	'CrdR',	'2CrdY'	, 'Fls', 'Fld',	'Off', 'PKwon',	'PKcon', 'OG'], inplace = True)

In [330]:
matches_rolling.shape

(3660, 51)

In [331]:
matches_rolling

,Date,Time,Round,Day,Venue,Opponent,Season,Team,Result_rolling,GF_rolling,GA_rolling,Poss_rolling,Attendance_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling,SoTA_rolling,PKA_rolling,PKsv_rolling,PKm_rolling,Cmp_Total_rolling,Att_Total_rolling,TotDist_rolling,PrgDist_rolling,TI_rolling,CK_rolling,SCA_rolling,GCA_rolling,Tkl_Defensive_rolling,TklW_Defensive_rolling,Blocks_Defensive_rolling,Touches_rolling,Def Pen_rolling,Def 3rd_rolling,Mid 3rd_rolling,Att 3rd_rolling,Dribbles_Succ_rolling,Dribles_Att_rolling,Rec_rolling,CrdY_rolling,CrdR_rolling,2CrdY_rolling,Fls_rolling,Fld_rolling,Off_rolling,PKwon_rolling,PKcon_rolling,OG_rolling
3235,2017-09-25,20,6,1,0,25,2017,0,1.0,1.4,1.6,60.6,48558.4,16.2,5.4,17.50,0.2,0.0,0.0,4.6,0.0,0.0,0.0,526.2,644.6,8902.2,3081.6,20.2,6.4,30.2,2.4,17.2,13.2,7.2,747.0,54.0,192.2,360.6,201.2,11.0,15.2,522.8,1.4,0.0,0.0,11.6,8.6,3.8,0.0,0.0,0.0
3236,2017-10-01,12,7,7,0,4,2017,0,1.0,1.0,1.0,60.8,48507.8,13.8,4.4,17.38,0.8,0.2,0.2,4.4,0.0,0.0,0.0,526.2,640.4,8894.8,3118.8,19.8,6.0,25.4,1.4,15.6,11.6,7.0,741.2,54.0,185.8,361.6,200.4,10.4,15.2,522.8,1.6,0.0,0.0,11.4,9.6,2.8,0.2,0.0,0.0
3237,2017-10-14,17,8,6,1,24,2017,0,1.4,1.4,0.8,58.4,54491.6,15.2,4.8,17.44,1.2,0.2,0.2,3.8,0.0,0.0,0.0,518.6,630.2,8672.8,2997.0,19.0,5.4,27.2,2.0,15.6,11.6,8.0,734.6,57.8,184.6,353.2,202.8,9.2,13.8,516.0,1.6,0.0,0.0,10.6,10.2,2.6,0.2,0.0,0.0
3238,2017-10-22,13,9,7,1,9,2017,0,1.4,1.6,0.4,58.8,47927.2,15.4,6.0,18.24,1.2,0.2,0.2,2.2,0.2,0.0,0.0,518.4,631.8,8820.4,3048.0,18.4,5.8,27.4,2.2,16.6,11.8,9.0,736.8,57.8,189.0,347.6,205.4,7.8,11.6,515.0,0.8,0.0,0.0,10.8,10.6,1.6,0.2,0.2,0.0
3239,2017-10-28,15,10,6,0,22,2017,0,1.4,2.0,0.8,60.6,43912.6,18.0,7.0,18.44,1.4,0.2,0.2,2.4,0.2,0.0,0.0,529.4,639.6,8944.6,3106.8,19.2,5.2,31.2,3.2,15.8,11.2,9.8,744.4,55.6,193.8,349.4,205.8,8.4,11.8,525.8,1.0,0.0,0.0,10.8,11.6,1.0,0.2,0.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2022-04-30,15,34,6,0,4,2021,27,0.8,1.0,1.2,56.0,34675.2,10.8,4.2,18.96,0.0,0.0,0.0,4.2,0.4,0.0,0.0,452.8,552.8,8124.6,2679.2,20.2,3.2,19.0,1.6,20.4,11.2,10.6,655.6,64.6,230.6,304.0,127.0,10.4,20.2,447.6,2.0,0.2,0.2,11.8,9.2,0.8,0.0,0.4,0.0
376,2022-05-07,15,35,6,1,7,2021,27,0.4,0.8,1.8,53.6,33101.4,10.2,3.8,18.38,0.2,0.0,0.0,5.6,0.6,0.0,0.2,423.4,526.6,7495.4,2532.8,20.6,4.2,17.2,1.2,21.8,12.2,11.6,625.8,66.8,236.2,278.0,117.6,11.2,21.6,418.2,2.2,0.2,0.2,10.8,7.6,1.0,0.0,0.8,0.0
377,2022-05-11,20,36,3,0,15,2021,27,0.6,0.8,1.6,52.2,33171.0,10.2,3.0,17.32,0.2,0.0,0.0,5.8,0.8,0.0,0.2,408.6,510.4,7092.2,2428.2,19.0,4.4,16.4,1.4,22.4,12.6,10.4,611.2,70.2,235.2,270.8,111.2,11.0,19.0,403.4,2.2,0.0,0.0,10.4,6.2,1.0,0.0,1.0,0.0
378,2022-05-15,14,37,7,0,18,2021,27,0.2,0.6,2.4,49.2,33368.6,9.4,2.8,17.70,0.2,0.0,0.0,5.8,0.6,0.0,0.2,404.4,502.6,7065.4,2395.0,16.6,4.8,15.8,1.4,21.0,12.2,9.4,599.4,71.0,233.4,266.2,104.8,11.0,19.4,399.6,1.4,0.0,0.0,7.8,6.4,1.2,0.0,0.8,0.0


In [332]:
matches_rolling.to_csv('EPL_2017-2021.csv', mode='a', index=False, header=True)